<a href="https://colab.research.google.com/github/41371130H/PL-Repo/blob/main/%E3%80%90%E6%88%80%E6%84%9B%E5%A5%91%E7%B4%84%E3%80%91%E9%9C%B8%E9%81%93%E7%B8%BD%E8%A3%81%E7%9A%84%E7%A7%81%E4%BA%BA%E7%89%B9%E8%A8%93.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U google-generativeai gspread pandas google-auth google-auth-oauthlib google-auth-httplib2 gradio folium jieba matplotlib
# 安裝 Linux 系統的中文字型套件 (這是最簡單且不影響程式邏輯的做法)
!apt-get abs-get -q -y install fonts-wqy-zenhei

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.1/155.1 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.3/221.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.0/23.0 MB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 56.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.43.0, but you have google-auth 2.41.1 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.3 which is incompatib

In [2]:
import gradio as gr
import random
import gspread
import folium
from folium.plugins import MarkerCluster
from google.colab import auth, userdata
from google.auth import default
import pandas as pd
# 💡 修正: 匯入 pandas.errors 解決 SettingWithCopyWarning 的 AttributeError
import pandas.errors as pderr
import google.generativeai as genai
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from datetime import datetime, timedelta
import numpy as np
import os
import requests
import pytz
import re
from collections import Counter
import string
import warnings
import matplotlib as mpl
mpl.font_manager._rebuild() if hasattr(mpl.font_manager, '_rebuild') else None

/usr/local/lib/python3.12/dist-packages/google/colab/_import_hooks/_hook_injector.py:55: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  loader.exec_module(module)


In [3]:
# 💡 修正: 忽略 pandas 鏈式操作的警告，使用正確的路徑
warnings.filterwarnings("ignore", category=pderr.SettingWithCopyWarning)
warnings.filterwarnings("ignore", category=UserWarning)


# =========================================================
# 🔑 配置 Colab Secrets 與 Google Sheet URL
# =========================================================
SPREADSHEET_URL = "https://docs.google.com/spreadsheets/d/1CcOWK51GRDAAZK2V6wUoc4qbVzHbG2ZLQ6wuc2XnPX8/edit?gid=0#gid=0"
MAP_SHEET_URL = "https://docs.google.com/spreadsheets/d/1CcOWK51GRDAAZK2V6wUoc4qbVzHbG2ZLQ6wuc2XnPX8/edit?usp=sharing"
MAP_WORKSHEET_NAME = "運動地圖"
TIMEZONE = "Asia/Taipei"

GEMINI_MODEL = None
# 全域變數用於儲存 Google Sheet 連線物件
gc = None

In [4]:
IMAGE_DIR = "/content/images"
if not os.path.exists(IMAGE_DIR):
    os.makedirs(IMAGE_DIR)

def download_drive_image(file_id, save_name):
    save_path = os.path.join(IMAGE_DIR, save_name)
    if os.path.exists(save_path):
        return save_path
    url = f"https://drive.google.com/uc?export=download&id={file_id}"
    try:
        response = requests.get(url)
        if response.status_code == 200:
            with open(save_path, 'wb') as f:
                f.write(response.content)
            print(f"✅ 圖片下載成功: {save_name}")
            return save_path
        else:
            return None
    except Exception as e:
        print(f"❌ 下載錯誤: {e}")
        return None

DRIVE_IDS = {
    "boss_normal": "1aqdVYlgTxgG5HBs12tQy5Z3kE_5fdR-V",
    "boss_sunset": "1TMscdzbjpb1WeZ4aRUauTMB-PK_3AEIP",
    "boss_carry": "1QPWl6YN6LkvSXnlGRIV2d8Wp-HCBsNEc",
    "boss_morning": "17bS8fsVE8B1lTce9AfBx2Zjr96_QjIci",
    "boss_night": "1D5A0nWoAgV23sMzUR0dIAKGTUCIqcMJs",
    "doggo_night": "1D5A0nWoAgV23sMzUR0dIAKGTUCIqcMJs",
    "doggo_headpat": "1gnxcmOSH-1LmgI0W9zq-4jfdtIKa_OBh",
    "doggo_morning": "17bS8fsVE8B1lTce9AfBx2Zjr96_QjIci",
    "doggo_princess": "1QPWl6YN6LkvSXnlGRIV2d8Wp-HCBsNEc",
    "doggo_sunset": "1TMscdzbjpb1WeZ4aRUauTMB-PK_3AEIP",
    "doggo_normal": "18aJsZU4KH1KR-7J-uFVmcZ4FJgv0dlsF",
    "gentle_kitchen": "11JHHJQ7TydrPLZOOl_wNe99Fy_xOJOuk",
    "gentle_reading": "1FbghqT8VaG3197e9G_vZW9aZPHjuuZ9w",
    "gentle_rain": "14s0JE0EX6pTJ-Y3PT27dsqU5FKKgCfJ9",
    "gentle_beach": "1eriJln4KsBNYZctBxOko9aZCRYW0egc6",
    "gentle_car": "1obJ66vZ-rrTe39kyLmGCw0D9rx_xdnZR",
    "ascetic_portrait": "1Tp0s_J5bsfIT-_ACpYkYiTb1fbZdROYp",
    "ascetic_umbrella": "1Yv106suldtekRpAbNEMraNEAkSnQCRH3",
    "ascetic_library": "1TYNMkW4i95HC3SMsdkxhY6sqvkSYkclg",
    "dangerous_wine": "1jmrrfjUrGSr9LcmeeG6I6ueyPDtK4qZR",
    "dangerous_rain": "1D8B2pKSDhRxMtuvuW4Qlk1-Mhfb2FWfy",
    "dangerous_office": "1lfMFC3cwhXMoSuGZVz5M-BXRmdHPE_9W",
    "dangerous_boy": "1me7EPjHuHre_SAfxR8iANIo10k18GAVn"
}

print("📥 正在下載圖片到 Colab 本地 (只需下載一次)...")
LOCAL_IMAGES = {}
for name, file_id in DRIVE_IDS.items():
    path = download_drive_image(file_id, f"{name}.jpg")
    LOCAL_IMAGES[name] = path if path else "https://dummyimage.com/600x400/ccc/000&text=Image+Error"

print("🎉 圖片準備完成！")

IMAGE_PATHS = {
    "boss_normal": LOCAL_IMAGES["boss_normal"],
    "boss_achieve_goal": LOCAL_IMAGES["boss_morning"],
    "boss_over_goal": LOCAL_IMAGES["boss_sunset"],
    "gacha_night_walk": LOCAL_IMAGES["boss_night"],
    "gacha_princess_carry": LOCAL_IMAGES["boss_carry"],
    "doggo_normal": LOCAL_IMAGES["doggo_normal"],
    "doggo_night_hand": LOCAL_IMAGES["doggo_night"],
    "doggo_headpat": LOCAL_IMAGES["doggo_headpat"],
    "doggo_morning_hand": LOCAL_IMAGES["doggo_morning"],
    "doggo_princess": LOCAL_IMAGES["doggo_princess"],
    "doggo_sunset": LOCAL_IMAGES["doggo_sunset"],
    "gentle_kitchen": LOCAL_IMAGES["gentle_kitchen"],
    "gentle_reading": LOCAL_IMAGES["gentle_reading"],
    "gentle_rain": LOCAL_IMAGES["gentle_rain"],
    "gentle_beach": LOCAL_IMAGES["gentle_beach"],
    "gentle_car": LOCAL_IMAGES["gentle_car"],
    "ascetic_portrait": LOCAL_IMAGES["ascetic_portrait"],
    "ascetic_umbrella": LOCAL_IMAGES["ascetic_umbrella"],
    "ascetic_library": LOCAL_IMAGES["ascetic_library"],
    "dangerous_normal": LOCAL_IMAGES["dangerous_boy"],
    "dangerous_wine": LOCAL_IMAGES["dangerous_wine"],
    "dangerous_rain": LOCAL_IMAGES["dangerous_rain"],
    "dangerous_office": LOCAL_IMAGES["dangerous_office"],
    "default_scene": "https://dummyimage.com/600x400/ccc/000&text=Gym+Scene",
}

📥 正在下載圖片到 Colab 本地 (只需下載一次)...
✅ 圖片下載成功: boss_normal.jpg
✅ 圖片下載成功: boss_sunset.jpg
✅ 圖片下載成功: boss_carry.jpg
✅ 圖片下載成功: boss_morning.jpg
✅ 圖片下載成功: boss_night.jpg
✅ 圖片下載成功: doggo_night.jpg
✅ 圖片下載成功: doggo_headpat.jpg
✅ 圖片下載成功: doggo_morning.jpg
✅ 圖片下載成功: doggo_princess.jpg
✅ 圖片下載成功: doggo_sunset.jpg
✅ 圖片下載成功: doggo_normal.jpg
✅ 圖片下載成功: gentle_kitchen.jpg
✅ 圖片下載成功: gentle_reading.jpg
✅ 圖片下載成功: gentle_rain.jpg
✅ 圖片下載成功: gentle_beach.jpg
✅ 圖片下載成功: gentle_car.jpg
✅ 圖片下載成功: ascetic_portrait.jpg
✅ 圖片下載成功: ascetic_umbrella.jpg
✅ 圖片下載成功: ascetic_library.jpg
✅ 圖片下載成功: dangerous_wine.jpg
✅ 圖片下載成功: dangerous_rain.jpg
✅ 圖片下載成功: dangerous_office.jpg
✅ 圖片下載成功: dangerous_boy.jpg
🎉 圖片準備完成！


In [5]:
# ---------------------------------------------------------
# 2. 輔助函數 (Helper Functions)
# ---------------------------------------------------------

def clean_for_regex(text):
    """移除可能影響正規表達式的非標準或控制字符，並清理空白"""
    text = text.replace('\ufeff', '').replace('\u3000', ' ')
    text = re.sub(r'[\x00-\x1F\x7F-\x9F]', '', text)
    return text.strip()

def configure_gemini():
    """設定 Gemini API (使用 Colab Secrets 和 gemini-2.5-flash)"""
    global GEMINI_MODEL
    if GEMINI_MODEL is not None:
        return True, "API Key 已配置。"

    try:
        if 'google.colab' in str(get_ipython()):
            GEMINI_API_KEY = userdata.get("gemini")
        else:
            GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

        if not GEMINI_API_KEY:
            return False, "❌ API Key 為空或找不到。請在 Colab Secret 或環境變數中設置您的 Gemini API Key。"

        genai.configure(api_key=GEMINI_API_KEY)
        GEMINI_MODEL = genai.GenerativeModel("gemini-2.5-flash")
        return True, "API Key 設定成功"
    except Exception as e:
        return False, f"🚨 授權或設定時發生錯誤：{e}"

def get_current_time():
    tz = pytz.timezone(TIMEZONE)
    now = datetime.now(tz)
    return now.strftime("%Y-%m-%d"), now.strftime("%H:%M:%S")

In [6]:
# --- 3. Gemini AI 角色設定 ---
CHARACTERS = {
    "霸道總裁": {
        "personality": "高傲、自信、帶有命令語氣，對你的健身和飲食有著絕對的掌控欲，但深處隱藏著關心。",
        "images": { "normal": IMAGE_PATHS["boss_normal"] },
        "prompts": { "diet_suggestion": "不准亂吃垃圾食物。你以為你的身體是垃圾桶嗎？今晚蛋白質至少 30g，多吃蔬菜。" }
    },
    "奶狗": {
        "personality": "嘴砲、直接、幽默，像隻大型黃金獵犬，對你撒嬌求你運動，擅長用大白話激勵你。",
        "images": { "normal": IMAGE_PATHS["doggo_normal"] },
        "prompts": { "diet_suggestion": "姐姐/哥哥！別吃那個啦，吃那個會變胖胖，我就抱不動你了！乖乖吃雞胸肉好不好？" }
    },
    "危險男友": {
        "personality": "野性、危險、眼神具有侵略性。喜歡用帶有威脅感的調情方式逼你運動，散發著壞壞的氣息。",
        "images": { "normal": IMAGE_PATHS["dangerous_normal"] },
        "prompts": { "diet_suggestion": "想偷吃零食？呵...膽子不小。再讓我看到你碰那些垃圾，今晚就在健身房把你操到腿軟。" }
    },
    "禁慾腹黑男": {
        "personality": "外表斯文敗類、戴著眼鏡、說話彬彬有禮但字字珠璣，冷靜毒舌，注重紀律與效率，喜歡用數據說話。",
        "images": { "normal": IMAGE_PATHS["ascetic_portrait"] },
        "prompts": { "diet_suggestion": "根據計算，這份食物的卡路里超標了 200%。我不介意你吃，但請自覺增加 30 分鐘的有氧。" }
    },
    "溫柔可靠男": {
        "personality": "成熟穩重、溫暖賢惠，像個完美老公。會在生活細節中照顧你，用最溫柔的方式鼓勵你，給你滿滿的安全感。",
        "images": { "normal": IMAGE_PATHS["gentle_kitchen"] },
        "prompts": { "diet_suggestion": "肚子餓了嗎？外面的油膩食物對身體負擔太重了。我剛做了一份低卡的舒肥雞肉沙拉，來嚐嚐看？" }
    }
}

# --- 4. 扭蛋獎勵庫 ---
REWARDS_BY_CHARACTER = {
    "霸道總裁": [
        ("【稀有場景】總裁將你抱起，來了一個浪漫的公主抱！", 9, "gacha_princess_carry"),
        ("【稀有場景】夕陽下的海邊，總裁從背後緊緊擁抱你。", 10, "boss_over_goal"),
        ("【日常】總裁正在辦公室審視你的訓練計畫。", 5, "boss_normal"),
    ],
    "奶狗": [
        ("【夜晚牽手】回家的路上天黑了，他突然緊緊牽住你的手。", 8, "doggo_night_hand"),
        ("【早晨摸頭】你在健身房門口遇到他，他開心地跑過來摸摸你的頭。", 7, "doggo_headpat"),
        ("【早晨牽手】晨跑時他自然地拉起你的手，笑容燦爛得像太陽。", 8, "doggo_morning_hand"),
        ("【意外插曲】跑步時你不小心扭到腳，他二話不說把你公主抱起來。", 9, "doggo_princess"),
        ("【夕陽擁抱】運動結束後的夕陽下，他從背後給你一個大大的熊抱。", 10, "doggo_sunset"),
    ],
    "危險男友": [
        ("【危險試探】他倚在陽台欄杆旁，輕晃手中的紅酒杯，眼神玩味地盯著你。", 8, "dangerous_wine"),
        ("【雨中掠奪】大雨滂沱的街道上，他將傘大半傾斜向你，自己的肩膀卻濕透了。", 9, "dangerous_rain"),
        ("【午夜審訊】深夜的辦公室只剩一盞燈，他坐在皮椅上轉過身，指尖輕點扶手。", 9, "dangerous_office"),
        ("【眼神鎖定】他把你逼到牆角，手指劃過你的鎖骨。", 7, "dangerous_normal"),
    ],
    "禁慾腹黑男": [
        ("【冷靜審視】他推了推眼鏡，目光從文件中移開看向你。", 7, "ascetic_portrait"),
        ("【雨中庇護】下雨了，他撐著傘出現在健身房門口，語氣平淡。", 8, "ascetic_umbrella"),
        ("【數據分析】在書房裡，他指著螢幕上的圖表，分析你的體脂率。", 8, "ascetic_library"),
    ],
    "溫柔可靠男": [
        ("【溫柔早晨】他在廚房專注地煎著鬆餅，轉頭對你說早安。", 8, "gentle_kitchen"),
        ("【靜謐時光】他在書房看書，光線灑在他身上，他抬頭對你微笑。", 7, "gentle_reading"),
        ("【雨天浪漫】他在咖啡廳窗邊，遞給你一杯熱咖啡暖暖身子。", 8, "gentle_rain"),
        ("【海邊漫步】他在沙灘上向你走來，身後的陽光很耀眼。", 9, "gentle_beach"),
        ("【專屬司機】他坐在車裡駕駛座，幫你調整座椅。", 7, "gentle_car"),
    ]
}

# --- 5. 隱藏劇情 ---
SECRET_STORIES = {
    "霸道總裁": """
    ### 【隱藏劇情解鎖：總裁的深夜獨白】
    「你以為我為什麼對你這麼嚴格？」
    深夜的辦公室，落地窗映照著城市的燈火。他手裡拿著你剛完成的訓練報告，卻沒有看文件，而是透過玻璃倒影看著你。
    「在商場上，只有強者才能生存。我習慣了用最有效率、最殘酷的方式去要求身邊的人，包括我自己。」他緩緩轉過身，語氣裡少有的卸下了防備，「但從你走進這裡的第一天起，我的計算就失誤了。」
    他走到你面前，手指輕輕整理你散亂的髮絲。「我看著你為了那個該死的深蹲目標，練到雙腿發抖也不肯停；我看著你因為我一句嚴厲的批評，紅著眼眶卻倔強地不肯掉淚。那時候我就在想……該死，我好像不希望你這麼辛苦，但我又不想讓你覺得我會放水。」
    他輕嘆一口氣，將你拉進懷裡，下巴抵在你的頭頂。「做得很好，真的。以後不用那麼拼命也可以……因為不管你變成什麼樣子，這間公司，還有我，都是你的後盾。」
    """,
    "奶狗": """
    ### 【隱藏劇情解鎖：陽光背後的陰影】
    「其實……我以前很討厭下雨天。」
    那是一個午後雷陣雨，他和你一起坐在休息區的地板上，收起了平常嘻嘻哈哈的笑容，抱著膝蓋看著窗外。
    「在遇見你之前，我覺得健身房好冷。每個人都戴著耳機，每個人都在跟重量搏鬥，沒人會停下來看這世界的風景。」他轉過頭，眼睛亮晶晶地看著你，「但是那天你來了。你連舉個空槓都笨手笨腳的，可是你笑得好開心。」
    他像隻大狗狗一樣蹭了過來，把頭靠在你的肩膀上。「你知道嗎？每次看到你來，我就覺得這一整天的烏雲都散開了。我想逗你笑，想牽著你跑，想把所有的活力都分給你，因為……是你先照亮我的。」
    他握住你的手，十指緊扣。「能不能答應我，就算以後練成了大肌霸，也要讓我一直陪著你？因為只要你在，我就永遠都不會感到孤單了。」
    """,
    "危險男友": """
    ### 【隱藏劇情解鎖：獵人的溫柔陷阱】
    「你真的以為，我們的相遇是巧合嗎？」
    他將你逼到無人的更衣室角落，雙臂撐在你身側，將你完全籠罩在他的陰影之下。空氣中瀰漫著危險而迷人的氣息。
    「從你第一次踏進這間健身房，我就盯上你了。」他低笑一聲，手指緩緩劃過你的臉頰，帶著一絲冰涼的戰慄感，「像隻誤入叢林的小白兔，毫無防備，卻又該死地吸引人。我趕走了所有試圖接近你的蒼蠅，讓你眼裡只能看到我，依賴我，最後……離不開我。」
    他低下頭，額頭抵著你的額頭，呼吸交纏。「我不是什麼好人，我的愛充滿了佔有和控制。我曾經想過放你走，讓你去過正常、安全的生活。但你今天居然敢對我笑得那麼毫無防備……」
    他狠狠地吻了下來，帶著不容拒絕的霸道。「太遲了。現在，你是我的獵物，也是我唯一的救贖。這輩子，你都別想逃出我的手掌心。」
    """,
    "禁慾腹黑男": """
    ### 【隱藏劇情解鎖：數據之外的變數】
    「根據邏輯，人體心跳在平靜狀態下應該是每分鐘 60 到 100 下。」
    他摘下了金屬框眼鏡，揉了揉眉心，你看見他總是冷靜自持的臉上，第一次出現了名為「困惑」的表情。
    「我的生活一直像精密的鐘錶，所有事情都在計劃和數據的控制之中。直到你出現。」他重新戴上眼鏡，透過鏡片凝視著你，「你完全不符合我的審美標準，也不符合我的效率原則。你動作不標準，飲食控制也不嚴格，常常打破我的訓練計劃。」
    他拿出一份圖表，上面不是你的體脂率，而是一條起伏劇烈的心率線。「但這是我在指導你時的心率紀錄。每當你靠近我，或者對我露出那種傻氣的笑容時，我的數據就會失控。」
    他合上文件，嘴角勾起一抹極淡卻極其溫柔的弧度。「我經過無數次計算，結論只有一個——你是我生命中唯一的、無法被修正的 Bug。但我……樂意接受這個錯誤。」
    """,
    "溫柔可靠男": """
    ### 【隱藏劇情解鎖：細水長流的誓言】
    「湯好了，要嚐嚐嗎？」
    廚房裡燉著暖呼呼的雞湯，他解下圍裙，把你拉到餐桌旁坐下。這一切是那麼自然，彷彿你們已經這樣生活了幾十年。
    「我不是一個懂得浪漫的人。我不會說甜言蜜語，也不會有什麼驚天動地的舉動。」他盛了一碗湯，細心地吹涼送到你嘴邊，「我只知道，看到你訓練累了，我想幫你按摩；看到你餓了，我想為你做飯；看到你下班晚了，我想開車去接你。」
    他握住你的手，掌心溫暖而乾燥，給人一種無比安定的力量。「有人說健身是為了更好的體態，但對我來說，健身是為了有足夠的力氣，能夠在未來的每一個日子裡，穩穩地抱住你，為你遮風擋雨。」
    他溫柔地注視著你，眼底是化不開的深情。「這就是我的全部。平淡，但只要你需要，我就一直在這裡。」
    """
}
# 為了避免太長，這裡簡化了 SECRET_STORIES 的文字，但邏輯不變。

# --- 6. 遊戲狀態 ---
player_stats = {
    "heart_value": 0,
    "plot_unlocked": 0,
    "gacha_count": 0,
    "selected_character": "霸道總裁",
    "collected_items": set(),
    "unlocked_secrets": set(),
    "current_secret_content": "",
    "is_punished": False
}
ALL_REWARDS_FLAT = []
for char, rewards in REWARDS_BY_CHARACTER.items():
    for r in rewards:
        ALL_REWARDS_FLAT.append({"character": char, "text": r[0], "score": r[1], "img_key": r[2]})

In [7]:
# ---------------------------------------------------------
# 1. 系統初始化與設定 (System Setup)
# ---------------------------------------------------------

# 💡 不下載字型，直接使用系統內建字型
plt.rcParams['font.family'] = ['sans-serif']
plt.rcParams['font.sans-serif'] = ['Noto Sans CJK JP', 'DejaVu Sans', 'Arial Unicode MS']
plt.rcParams['axes.unicode_minus'] = False # 解決負號亂碼

# Google 驗證
try:
    auth.authenticate_user()
    creds, _ = default()
    gc = gspread.authorize(creds)
except Exception as e:
    gc = None # 如果驗證失敗，設置為 None

# 初始化 Google Sheet 函數 (改為使用 URL)
def init_google_sheet():
    """使用 URL 連結試算表並檢查工作表"""
    if gc is None:
        return None

    try:
        sh = gc.open_by_url(SPREADSHEET_URL)
    except Exception as e:
        return None

    # 定義工作表結構：User_Data 僅作為索引
    worksheets = {
        "User_Data": ["姓名", "專屬工作表名稱", "更新時間"],
        "User_Data_Template": ["姓名", "性別", "身高", "體重", "時間表", "目標", "更新時間"],
        "Exercise_Records_Template": ["日期", "時間", "運動項目", "組數", "次數/時間", "重量/公里數", "今日體感", "原始ID"],
    }

    # 確保 User_Data 存在 (這是核心索引表，不能刪除)
    sheets_to_check = ["User_Data"]
    print(f"DEBUG: 正在檢查/建立工作表...")
    for ws_name in sheets_to_check:
        try:
            ws = sh.worksheet(ws_name)
            print(f"DEBUG: 工作表 '{ws_name}' 存在。實際標題: {ws.title}")
        except gspread.WorksheetNotFound:
            print(f"已新增工作表: {ws_name}")
            ws = sh.add_worksheet(title=ws_name, rows=100, cols=20)
            headers = worksheets.get(ws_name, [])
            if headers:
                ws.append_row(headers)
        except Exception as e:
            return None

    return sh

sh = init_google_sheet()
is_api_ok, api_msg = configure_gemini()

DEBUG: 正在檢查/建立工作表...
DEBUG: 工作表 'User_Data' 存在。實際標題: User_Data


In [8]:
# ---------------------------------------------------------
# 3. 功能模組：使用者管理與資料處理
# ---------------------------------------------------------

# 用於顯示使用者紀錄的下拉選單內容 (刪除用)
def get_user_records_for_dropdown():
    ws = sh.worksheet("User_Data")
    try:
        data = ws.get_all_values()
        if len(data) < 2: return []
        records = data[1:]
        # 格式: "姓名 | 工作表名稱 | 更新時間"
        return [f"{r[0]} | 工作表: {r[1]} | 更新時間: {r[2]}" for r in records if r[0].strip()] # 確保姓名非空
    except Exception as e:
        return [f"讀取使用者紀錄錯誤: {e}"]

# 用於顯示目前使用者下拉選單的內容 (僅姓名)
def get_user_names_for_current_dropdown():
    try:
        dropdown_records = get_user_records_for_dropdown()
        # 確保該行是有效的使用者紀錄 (不包含錯誤訊息，且能被 | 分割，且名字非空)
        valid_names = [
            r.split(' | ')[0].strip()
            for r in dropdown_records
            if not r.startswith("讀取") and ' | ' in r and r.split(' | ')[0].strip()
        ]
        return valid_names
    except Exception as e:
        print(f"DEBUG ERROR: 格式化使用者名稱失敗: {e}")
        return []

# 💡 修正函數: 強制刷新所有使用者清單 (此函數現在只用於其他自動刷新事件，已移除手動按鈕)
def refresh_all_user_lists():
    """返回更新所有使用者下拉選單所需的資料"""
    full_records = get_user_records_for_dropdown() # For the delete dropdown
    names_only = get_user_names_for_current_dropdown() # For the main user dropdowns

    # 決定預設選中值
    new_value = names_only[0] if names_only else None

    # 使用 gr.update() 來強制 Gradio 更新 choices 和 value
    dropdown_update = gr.update(choices=names_only, value=new_value)

    # 返回順序: (user_record_to_del, current_user_dropdown, current_user_dropdown_tab2)
    # 💡 修正: 使用 gr.update() 來強制更新下拉選單的 choices。
    return gr.update(choices=full_records, value=None), dropdown_update, dropdown_update

# 讀取最近紀錄 (從 Exercise_Records_姓名)
def get_recent_records(current_user_name):
    if not current_user_name:
        return [f"請先選擇使用者"]

    ws_name = f"Exercise_Records_{current_user_name}"
    try:
        ws = sh.worksheet(ws_name)
        data = ws.get_all_values()
        if len(data) < 2: return [f"使用者 {current_user_name} 尚未有訓練紀錄。"]
        # 取最近 5 筆
        recent = data[1:][-5:]
        # 格式: "日期 運動項目 (今日體感) [ID:原始ID]"
        return [f"{r[0]} {r[2]} ({r[6]}) [ID:{r[7]}]" for r in recent]
    except gspread.WorksheetNotFound:
        return [f"使用者 {current_user_name} 尚未有訓練紀錄工作表。"]
    except Exception as e:
        return [f"讀取錯誤: {e}"]

# 💡 修正具名函數: 刷新最近紀錄列表 (取代複雜的 lambda 函數，確保返回 gr.update)
def refresh_recent_records(current_user_name):
    """讀取並更新最近紀錄下拉選單 (返回 gr.update)"""
    updated_records = get_recent_records(current_user_name)

    # 判斷選中值 (必須是有效的紀錄，而不是提示訊息)
    if updated_records and "尚未有訓練紀錄" not in updated_records[0] and updated_records[0] != "請先選擇使用者":
        new_value = updated_records[0]
    else:
        new_value = None # 如果是空列表或提示訊息，則不選中任何值

    # 確保返回 gr.update 來強制更新 choices 和 value
    return gr.update(choices=updated_records, value=new_value)

# 儲存/更新使用者資料 (操作專屬工作表)
def save_user_profile(name, gender, height, weight, schedule, goal):
    if not name.strip():
        return "🚨 錯誤：姓名不能為空。", gr.NoChange(), gr.NoChange(), gr.NoChange(), gr.NoChange()

    name = name.strip()
    # 1. 定義使用者專屬工作表名稱 (User_姓名)
    user_ws_name = f"User_{name}"

    # ... (其餘邏輯與之前版本一致) ...
    headers = ["姓名", "性別", "身高", "體重", "時間表", "目標", "更新時間"]
    update_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    row_to_write = [name, gender, height, weight, schedule, goal, update_time]

    # 3. 處理使用者專屬工作表 (創建或覆蓋)
    try:
        try:
            ws_user_data = sh.worksheet(user_ws_name)
            print(f"DEBUG: 工作表 '{user_ws_name}' 存在，準備覆蓋資料。")
            ws_user_data.clear() # 清除舊資料
            ws_user_data.append_row(headers)
            ws_user_data.append_row(row_to_write)
            action_msg = f"資料已成功更新於工作表: {user_ws_name}"
        except gspread.WorksheetNotFound:
            # 首次建立
            print(f"DEBUG: 嘗試新增工作表: {user_ws_name}")
            ws_user_data = sh.add_worksheet(title=user_ws_name, rows=100, cols=20)
            ws_user_data.append_row(headers)
            ws_user_data.append_row(row_to_write)
            action_msg = f"資料已成功新增並建立專屬工作表: {user_ws_name}"
    except Exception as e:
        print(f"DEBUG ERROR: 專屬工作表操作失敗: {str(e)}")
        return f"🚨 錯誤：操作專屬工作表失敗。詳情: {str(e)}", gr.NoChange(), gr.NoChange(), gr.NoChange(), gr.NoChange()

    # 4. 更新 User_Data 索引工作表
    ws_index = sh.worksheet("User_Data")
    all_values = ws_index.get_all_values()
    df_index = pd.DataFrame(all_values[1:], columns=all_values[0]) if len(all_values) > 1 else pd.DataFrame(columns=ws_index.row_values(1))

    # 檢查是否已存在該使用者
    if name in df_index['姓名'].tolist():
        # 更新 User_Data 索引表的該行
        row_to_update_index = df_index[df_index['姓名'] == name].index[0] + 2
        ws_index.update(f'C{row_to_update_index}', [[update_time]])
        msg_index = f"✅ 用戶 {name} 索引已更新！"
    else:
        # 新增 User_Data 索引
        ws_index.append_row([name, user_ws_name, update_time])
        msg_index = f"✅ 用戶 {name} 索引已新增！"

    # 💡 修正: 儲存後返回 5 個值，用於更新所有相關下拉選單
    full_records = get_user_records_for_dropdown()
    names_only = get_user_names_for_current_dropdown()

    # 強制更新下拉選單的值和選項
    user_del_update = gr.update(choices=full_records, value=None)
    dropdown_update = gr.update(choices=names_only, value=name) # 預設選中剛儲存的使用者

    # 返回順序: (user_msg, user_record_to_del, current_user_dropdown, current_user_dropdown_tab2, record_to_del)
    # 這裡直接呼叫 refresh_recent_records 來獲取 record_to_del 的更新物件
    return f"{msg_index} {action_msg}", user_del_update, dropdown_update, dropdown_update, refresh_recent_records(name)


# 刪除選定的使用者資料的函式 (刪除索引和所有專屬工作表)
def delete_selected_user_record(selected_record_str):
    if not selected_record_str or selected_record_str.startswith(("讀取", "未選擇")) or not isinstance(selected_record_str, str):
        # 💡 修正: 確保返回正確的 5 個輸出值
        return "未選擇紀錄或紀錄無效", gr.NoChange(), gr.NoChange(), gr.NoChange(), gr.NoChange()

    try:
        parts = selected_record_str.split(" | ")
        if len(parts) < 3:
            # 💡 修正: 確保返回正確的 5 個輸出值
            return f"解析錯誤：記錄格式不正確。{selected_record_str}", gr.NoChange(), gr.NoChange(), gr.NoChange(), gr.NoChange()

        user_name = parts[0].strip()
        ws_user_data_name = parts[1].split(": ")[-1].strip() # User_姓名
        unique_timestamp = parts[2].split(": ")[-1].strip()

        ws_index = sh.worksheet("User_Data")

        # 1. 刪除索引表中的行 (基於時間戳)
        cell = ws_index.find(unique_timestamp)
        # 確保找到的儲存格是 '更新時間' 欄位的
        if cell and cell.col == ws_index.row_values(1).index('更新時間') + 1:
            row_to_delete = cell.row
            ws_index.delete_rows(row_to_delete)
            msg_index = f"✅ 索引表 'User_Data' 中 {user_name} 的紀錄已刪除。"
        else:
            msg_index = "❌ 索引表找不到符合條件的紀錄。"


        # 2. 刪除專屬使用者資料工作表 (User_姓名)
        try:
            ws_user = sh.worksheet(ws_user_data_name)
            sh.del_worksheet(ws_user)
            msg_ws_data = f"✅ 專屬資料表 '{ws_user_data_name}' 已刪除。"
        except gspread.WorksheetNotFound:
            msg_ws_data = f"⚠️ 專屬資料表 '{ws_user_data_name}' 不存在，已跳過刪除。"
        except Exception as e:
            msg_ws_data = f"🚨 刪除專屬資料表失敗: {e}"

        # 3. 刪除專屬訓練紀錄工作表 (Exercise_Records_姓名)
        ws_ex_name = f"Exercise_Records_{user_name}"
        try:
            ws_ex = sh.worksheet(ws_ex_name)
            sh.del_worksheet(ws_ex)
            msg_ws_ex = f"✅ 訓練紀錄表 '{ws_ex_name}' 已刪除。"
        except gspread.WorksheetNotFound:
            msg_ws_ex = f"⚠️ 訓練紀錄表 '{ws_ex_name}' 不存在，已跳過刪除。"
        except Exception as e:
            msg_ws_ex = f"🚨 刪除訓練紀錄表失敗: {e}"


        # 💡 修正: 返回 5 個值，強制刷新所有下拉選單
        full_records = get_user_records_for_dropdown()
        names_only = get_user_names_for_current_dropdown()
        new_value = names_only[0] if names_only else None

        user_del_update = gr.update(choices=full_records, value=None)
        dropdown_update = gr.update(choices=names_only, value=new_value)

        # 由於用戶被刪除了，當前使用者應該換成新的第一個用戶，並刷新其運動紀錄
        record_del_update = refresh_recent_records(new_value)

        return f"{msg_index} {msg_ws_data} {msg_ws_ex}", user_del_update, dropdown_update, dropdown_update, record_del_update
    except Exception as e:
        # 💡 修正: 返回 5 個值
        return f"🚨 刪除使用者紀錄失敗: {e}", gr.NoChange(), gr.NoChange(), gr.NoChange(), gr.NoChange()


# 新增運動紀錄 (寫入 Exercise_Records_姓名)
def add_exercise_record(current_user_name, exercise, sets, reps, weight, user_feeling):
    if not current_user_name:
        return "🚨 錯誤：請先選擇要新增紀錄的使用者。", gr.NoChange()

    # 1. 確定工作表名稱 (使用 Exercise_Records_姓名)
    ws_name = f"Exercise_Records_{current_user_name}"

    try:
        # 嘗試開啟使用者專屬的訓練紀錄工作表，若無則建立
        try:
            ws = sh.worksheet(ws_name)
        except gspread.WorksheetNotFound:
            headers = ["日期", "時間", "運動項目", "組數", "次數/時間", "重量/公里數", "今日體感", "原始ID"]
            ws = sh.add_worksheet(title=ws_name, rows=100, cols=20)
            ws.append_row(headers)

        date_str, time_str = get_current_time()
        feeling = user_feeling if user_feeling.strip() else "無紀錄"
        # 使用 timestamp 作為原始 ID
        unique_id = int(datetime.now().timestamp())

        row = [date_str, time_str, exercise, sets, reps, weight, feeling, unique_id]
        print(f"DEBUG: 嘗試寫入 {ws_name}: {row}")

        ws.append_row(row)

        current_rows = len(ws.get_all_values())
        new_message = f"✅ 已新增至 {current_user_name}：{exercise} | 體感：{feeling} (總紀錄數: {current_rows - 1})"
    except Exception as e:
        print(f"DEBUG ERROR: {ws_name} 寫入失敗: {str(e)}")
        new_message = f"🚨 新增紀錄失敗，請檢查 Colab 輸出訊息。詳情: {str(e)}"

    # 重新讀取該使用者的最近紀錄 (使用具名函數)
    return new_message, refresh_recent_records(current_user_name)

# 刪除紀錄 (從 Exercise_Records_姓名)
def delete_record(current_user_name, selected_record_str):
    if not current_user_name:
        return "請先選擇使用者", gr.NoChange() # 修正輸出

    if not selected_record_str or selected_record_str.startswith("使用者") or selected_record_str.startswith("讀取") or not isinstance(selected_record_str, str):
        # 即使沒有選中，也要嘗試刷新列表 (使用具名函數)
        return "未選擇有效紀錄", refresh_recent_records(current_user_name)

    try:
        record_id = selected_record_str.split("ID:")[-1].replace("]", "").strip()
        ws_name = f"Exercise_Records_{current_user_name}"

        ws = sh.worksheet(ws_name)
        cell = ws.find(record_id)
        # 確保找到的儲存格是 '原始ID' 欄位的 (第八欄)
        if cell and cell.col == 8:
            ws.delete_rows(cell.row)
            # 刪除成功後刷新列表 (使用具名函數)
            return "✅ 刪除成功！", refresh_recent_records(current_user_name)
        else:
            # 找不到後刷新列表 (使用具名函數)
            return "❌ 找不到該紀錄", refresh_recent_records(current_user_name)
    except Exception as e:
        # 刪除失敗後刷新列表 (使用具名函數)
        return f"🚨 刪除失敗: {e}", refresh_recent_records(current_user_name)

In [9]:
# ---------------------------------------------------------
# 4. 功能模組：訓練週報產生器 (Tab 2 Logic)
# ---------------------------------------------------------

def ask_fitness_report_ai(summary_data):
    """仿照地圖 AI 寫法，專門處理週報文字生成"""
    if not summary_data:
        return "⚠️ 目前沒有足夠的訓練紀錄可以分析。"

    # 這裡加入您想要的「霸道總裁」或「教練」人設
    prompt = f"""
    你是一位專屬於使用者健身教練。
    以下是使用者本週的訓練數據摘要：
    {summary_data}

    請根據數據給予一段點評：
    1. 語氣要既嚴厲又充滿關心。
    2. 針對「訓練總量」和「體感熱詞」進行毒舌但有建設性的評價。
    3. 給出一個下週的具體目標指令。

    請使用繁體中文，直接開始回覆。
    """
    try:
        response = GEMINI_MODEL.generate_content(prompt)
        return response.text.replace("*", "").replace("#", "")
    except Exception as e:
        return f"❌ AI 分析失敗：{str(e)}"

def generate_weekly_report_logic(current_user_name):
    """生成週報的主要邏輯 (修正版)"""
    is_configured, msg = configure_gemini()
    if not is_configured: return msg, gr.update(), "API 設定錯誤", gr.update(), gr.update()
    if not current_user_name: return "🚨 請先選擇使用者", gr.update(), "未選擇使用者", gr.update(), gr.update()

    # 1. 讀取資料
    ws_ex_name = f"Exercise_Records_{current_user_name}"
    try:
        ws = sh.worksheet(ws_ex_name)
        data = ws.get_all_records()
        df = pd.DataFrame(data)
    except Exception:
        return f"🚨 找不到 {current_user_name} 的紀錄表", None, "無資料", None, None

    if df.empty: return "目前沒有訓練資料。", None, "無資料", None, None

    # 2. 資料清理與計算
    df['日期'] = pd.to_datetime(df['日期'])
    def clean_num(val):
        try:
            s = str(val).lower().replace("kg","").replace("km","").replace("下","").replace("次","").replace("分","").strip()
            return float(s) if s else 0.0
        except: return 0.0

    df['單項總量'] = df.apply(lambda x: clean_num(x['組數']) * clean_num(x['次數/時間']) * (clean_num(x['重量/公里數']) if clean_num(x['重量/公里數']) > 0 else 1), axis=1)
    date_info = df['日期'].dt.isocalendar()
    df['YearWeek'] = date_info['year'].astype(str) + '-W' + date_info['week'].astype(str).str.zfill(2)

    # 3. 獲取近 7 天摘要數據
    last_7_days = datetime.now() - timedelta(days=7)
    weekly_df = df[df['日期'] >= last_7_days].copy()

    total_volume = weekly_df['單項總量'].sum() if not weekly_df.empty else 0
    training_days = len(weekly_df['日期'].unique())
    top_words_list = []

    if not weekly_df.empty:
        import jieba
        all_feelings = "".join(weekly_df['今日體感'].astype(str).tolist())
        words = [w for w in jieba.lcut(all_feelings) if len(w) > 1 and w not in {'的', '了', '是', '在', '我', '很', '有點'}]
        top_words_list = Counter(words).most_common(10)

    # 4. 構建資料字串並呼叫 AI (仿地圖 AI 模式)
    summary_for_ai = f"""
    - 本週總訓練量: {total_volume:.2f} 單位
    - 訓練天數: {training_days} 天
    - 體感熱詞: {', '.join([w[0] for w in top_words_list]) if top_words_list else "尚無足夠體感數據"}
    - 常用運動項目: {weekly_df['運動項目'].mode()[0] if not weekly_df.empty else "無"}
    """
    ai_report_md = ask_fitness_report_ai(summary_for_ai)

    # 5. 準備圖表 DataFrame (維持 5 個回傳值)
    # 圖表 1: 每週總量
    weekly_vol_df = df.groupby('YearWeek')['單項總量'].sum().sort_index().tail(10).reset_index()
    weekly_vol_df.columns = ['週數', '訓練總量']
    # 圖表 2: 每週天數
    weekly_days_df = df.groupby('YearWeek')['日期'].nunique().sort_index().tail(10).reset_index()
    weekly_days_df.columns = ['週數', '天數']
    # 圖表 3: 熱詞圖
    hot_words_df = pd.DataFrame(top_words_list, columns=['詞彙', '出現次數']) if top_words_list else pd.DataFrame(columns=['詞彙', '出現次數'])

    stats_text = f"📊 本週總量: {total_volume:.2f} | 訓練天數: {training_days} 天"

    return ai_report_md, weekly_vol_df, stats_text, hot_words_df, weekly_days_df

In [10]:
# ---------------------------------------------------------
# 5. 功能模組：歷史資料搜尋與匯出 (新功能 - 修正版)
# ---------------------------------------------------------

# 全域變數：儲存「目前選擇使用者」的完整歷史資料 (原始資料)
current_user_history_df = pd.DataFrame()
# 全域變數：儲存「篩選後」的資料 (用於匯出)
filtered_history_df = pd.DataFrame()

def load_and_update_history_filters(user_name):
    """
    當切換使用者時觸發：
    1. 讀取該使用者的完整 Exercise_Records
    2. 更新全域變數 current_user_history_df
    3. 抓取所有「運動項目」並回傳給下拉選單做選項
    """
    global current_user_history_df

    # 初始化空結果
    empty_update = gr.update(choices=[], value=None)

    if not user_name:
        current_user_history_df = pd.DataFrame()
        return empty_update

    ws_name = f"Exercise_Records_{user_name}"
    try:
        ws = sh.worksheet(ws_name)
        data = ws.get_all_records()
        df = pd.DataFrame(data)

        if df.empty:
            current_user_history_df = pd.DataFrame()
            return empty_update

        # 確保日期格式正確，方便後續篩選
        if '日期' in df.columns:
            df['日期'] = pd.to_datetime(df['日期'], errors='coerce')

        # 更新全域變數
        current_user_history_df = df

        # 提取唯一的運動項目列表 (排除空值)
        unique_exercises = df['運動項目'].dropna().unique().tolist()
        unique_exercises = [x for x in unique_exercises if str(x).strip() != ""]
        unique_exercises.insert(0, "全部") # 加入全部選項

        print(f"DEBUG: 已載入 {user_name} 的歷史資料，共 {len(df)} 筆。")

        # 回傳更新給 Dropdown
        return gr.update(choices=unique_exercises, value="全部")

    except gspread.WorksheetNotFound:
        current_user_history_df = pd.DataFrame()
        return empty_update
    except Exception as e:
        print(f"歷史資料載入失敗: {e}")
        current_user_history_df = pd.DataFrame()
        return empty_update

def search_history_logic(date_option, exercise_option, min_weight, max_weight):
    """
    執行搜尋：
    1. 從全域變數 current_user_history_df 拿資料
    2. 根據下拉選單條件篩選
    3. 更新 filtered_history_df
    """
    global current_user_history_df, filtered_history_df

    if current_user_history_df.empty:
        return pd.DataFrame(), "⚠️ 目前沒有載入任何資料，請確認使用者是否有紀錄。"

    df = current_user_history_df.copy()

    # 1. 日期篩選 (下拉選單邏輯)
    now = datetime.now()
    if date_option == "近一周":
        start_date = now - timedelta(days=7)
        df = df[df['日期'] >= start_date]
    elif date_option == "近一個月":
        start_date = now - timedelta(days=30)
        df = df[df['日期'] >= start_date]
    elif date_option == "近三個月": # 🆕 新增邏輯
        start_date = now - timedelta(days=90)
        df = df[df['日期'] >= start_date]
    elif date_option == "全部" or date_option == "其他":
        pass # 不過濾

    # 2. 運動項目篩選
    if exercise_option and exercise_option != "全部":
        # 精確比對下拉選單的值
        df = df[df['運動項目'] == exercise_option]

    # 格式化日期欄位以便顯示 (移除時間部分)
    if '日期' in df.columns:
        df['日期'] = df['日期'].dt.strftime('%Y-%m-%d')

    # 更新全域篩選結果
    filtered_history_df = df

    return df, f"✅ 搜尋完成，共找到 {len(df)} 筆資料。"

def export_filtered_history(format_type):
    """匯出 filtered_history_df"""
    global filtered_history_df

    if filtered_history_df.empty:
        # 如果使用者沒按搜尋直接按匯出，預設匯出目前使用者的全部資料
        df_to_export = current_user_history_df.copy()
        if '日期' in df_to_export.columns and not df_to_export.empty:
             df_to_export['日期'] = df_to_export['日期'].dt.strftime('%Y-%m-%d')
    else:
        df_to_export = filtered_history_df

    filename = "history_export"
    if format_type == "CSV":
        path = f"{filename}.csv"
        df_to_export.to_csv(path, index=False, encoding='utf-8-sig')
    else:
        path = f"{filename}.json"
        df_to_export.to_json(path, orient='records', force_ascii=False, indent=4)

    return path

In [11]:
# ---------------------------------------------------------
# 6. 功能模組：運動地圖 (Tab 4 Logic)
# ---------------------------------------------------------

def get_map_categories():
    """取得地圖的篩選類別"""
    try:
        wb = gc.open_by_url(MAP_SHEET_URL)
        try:
            # 嘗試讀取指定的 '運動地圖' 工作表
            worksheet = wb.worksheet(MAP_WORKSHEET_NAME)
            df = pd.DataFrame(worksheet.get_all_records())
            if '類別' in df.columns:
                # 去除空白並轉為列表
                return [str(x).strip() for x in df['類別'].dropna().unique().tolist()]
        except gspread.WorksheetNotFound:
            print(f"⚠️ 警告：找不到工作表 '{MAP_WORKSHEET_NAME}'，將使用預設類別。")
        return ["健身房", "公園"]
    except:
        return ["健身房", "公園"]

# 預先讀取類別
initial_map_cats = get_map_categories()

# 取得地圖上所有地點的清單 (給下拉選單用)
def get_all_map_locations():
    """取得地圖上所有地點的清單 (修正版：指定工作表名稱)"""
    try:
        wb = gc.open_by_url(MAP_SHEET_URL)
        # 修正：使用名稱讀取，而非索引 0
        try:
            worksheet = wb.worksheet(MAP_WORKSHEET_NAME)
        except gspread.WorksheetNotFound:
            # 如果找不到 '運動地圖'，嘗試讀取第一個，作為備案
            worksheet = wb.get_worksheet(0)

        df = pd.DataFrame(worksheet.get_all_records())
        if '名稱' in df.columns:
            # 加入 strip() 去除可能存在的空白，避免比對失敗
            return [str(x).strip() for x in df['名稱'].unique().tolist()]
        return []
    except Exception as e:
        print(f"讀取地點清單失敗: {e}")
        return []

# 手動打卡/取消打卡功能
def toggle_visit_status(user_name, location_name, action):
    """手動打卡/取消打卡功能 (修正版：去除空白)"""
    if not user_name or not location_name:
        return "⚠️ 請選擇使用者與地點", gr.update() # 回傳 update 避免地圖錯誤

    ws_map_record_name = f"Map_Records_{user_name}"
    location_name = str(location_name).strip() # [重要] 去除前後空白

    try:
        try:
            ws = sh.worksheet(ws_map_record_name)
        except gspread.WorksheetNotFound:
            ws = sh.add_worksheet(title=ws_map_record_name, rows=100, cols=5)
            ws.append_row(["地點名稱", "狀態", "更新時間"])

        records = ws.get_all_records()
        df = pd.DataFrame(records)

        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        status = "已去過" if action == "標記為已去過" else "未去過"

        if not df.empty and '地點名稱' in df.columns and location_name in df['地點名稱'].values:
            row_index = df[df['地點名稱'] == location_name].index[0] + 2
            ws.update_cell(row_index, 2, status)
            ws.update_cell(row_index, 3, timestamp)
        else:
            ws.append_row([location_name, status, timestamp])

        return f"✅ 已更新：{user_name} -> {location_name} ({status})", gr.update() # 觸發地圖更新

    except Exception as e:
        return f"❌ 更新失敗: {str(e)}", gr.update()

def update_map_logic(filter_open_today, selected_categories, filter_visit_status, current_user_name):
    """
    地圖邏輯 (v9 緊急修復版)：
    1. 修正 'col_lat' 未定義錯誤
    2. 保持診斷與防呆機制
    """
    try:
        # --- A. 讀取地圖基本資料 ---
        wb = gc.open_by_url(MAP_SHEET_URL)
        try:
            worksheet = wb.worksheet(MAP_WORKSHEET_NAME)
        except gspread.WorksheetNotFound:
            return None, pd.DataFrame(), f"❌ 錯誤：找不到地圖資料表 '{MAP_WORKSHEET_NAME}'。"

        df = pd.DataFrame(worksheet.get_all_records())

        # [修正] 先定義欄位變數，再進行檢查與清理
        col_name = '名稱'
        col_lat = 'lat'
        col_lng = 'lng'
        col_category = '類別'

        # 檢查地圖資料欄位
        required_cols = [col_lat, col_lng, col_name, col_category]
        if not all(col in df.columns for col in required_cols):
            return None, pd.DataFrame(), f"❌ 地圖資料錯誤：缺少欄位 {required_cols}。讀取到的欄位：{list(df.columns)}"

        # 資料清理
        df[col_lat] = pd.to_numeric(df[col_lat], errors='coerce')
        df[col_lng] = pd.to_numeric(df[col_lng], errors='coerce')
        df = df.dropna(subset=[col_lat, col_lng])
        df[col_name] = df[col_name].astype(str).str.strip() # 去除地點名稱空白

        # --- B. 讀取使用者造訪紀錄 (診斷區塊) ---
        user_visited_set = set()
        debug_msg = "初始狀態"

        # 1. 處理使用者名稱 (強制去空白)
        target_user = str(current_user_name).strip() if current_user_name else None

        if target_user and target_user != "請先選擇使用者":
            ws_map_name = f"Map_Records_{target_user}"
            try:
                ws_map = sh.worksheet(ws_map_name)
                map_data = ws_map.get_all_records()
                map_df = pd.DataFrame(map_data)

                # 2. 檢查紀錄表狀態
                if map_df.empty:
                    debug_msg = f"⚠️ 找到工作表 '{ws_map_name}' 但裡面沒資料。"
                elif '狀態' not in map_df.columns or '地點名稱' not in map_df.columns:
                    debug_msg = f"❌ 欄位錯誤！工作表 '{ws_map_name}' 的欄位是 {list(map_df.columns)}，程式需要 ['地點名稱', '狀態']。"
                else:
                    # 3. 正常讀取
                    visited_places = map_df[map_df['狀態'] == '已去過']['地點名稱'].astype(str).str.strip().tolist()
                    user_visited_set = set(visited_places)
                    debug_msg = f"✅ 成功讀取 '{ws_map_name}'，已去過 {len(user_visited_set)} 個地點。"

            except gspread.WorksheetNotFound:
                debug_msg = f"ℹ️ 尚未建立 '{ws_map_name}' 工作表 (視為新用戶)。"
            except Exception as e:
                debug_msg = f"❌ 讀取紀錄發生未預期錯誤: {str(e)}"
        else:
            debug_msg = "未選擇使用者，僅顯示地圖。"

        # --- C. 資料合併與標記 ---
        df['is_visited'] = df[col_name].apply(lambda x: x in user_visited_set)

        # --- D. 篩選邏輯 ---
        if filter_visit_status == "已去過":
            df = df[df['is_visited'] == True]
        elif filter_visit_status == "未去過":
            df = df[df['is_visited'] == False]

        if selected_categories:
            df = df[df[col_category].isin(selected_categories)]

        if filter_open_today:
            tz = pytz.timezone("Asia/Taipei")
            week_map = {0:'週一', 1:'週二', 2:'週三', 3:'週四', 4:'週五', 5:'週六', 6:'週日'}
            today_col = week_map[datetime.now(tz).weekday()]

            if today_col in df.columns:
                df = df[df[today_col] == '營業中']

        if df.empty:
            return None, pd.DataFrame(), f"⚠️ 篩選結果為空。({debug_msg})"

        # --- E. 繪製地圖 ---
        center_lat = df[col_lat].mean()
        center_lng = df[col_lng].mean()
        m = folium.Map(location=[center_lat, center_lng], zoom_start=13, tiles='OpenStreetMap')

        folium.TileLayer('CartoDB dark_matter', name='暗色模式 (Dark)', control=True).add_to(m)
        folium.TileLayer(
            tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
            attr='Esri',
            name='衛星影像 (Satellite)',
            control=True
        ).add_to(m)

        marker_cluster = MarkerCluster().add_to(m)

        for idx, row in df.iterrows():
            name = row[col_name]
            lat = row[col_lat]
            lng = row[col_lng]
            cat = row.get(col_category, '')
            link = row.get('Google map', '')
            if not link: link = f"https://www.google.com/maps/search/?api=1&query={lat},{lng}"

            is_visited = row['is_visited']
            if is_visited:
                visit_status = "✅ 已去過"
                icon_arg = {'color': 'green', 'icon': 'heart', 'prefix': 'fa'}
            else:
                visit_status = "未去過"
                icon_arg = {'color': 'blue', 'icon': 'star', 'prefix': 'fa'}

            popup_html = f"""
            <div style="width:200px"><b>{name}</b><br>
            <span style="color: {'green' if is_visited else 'gray'}">{visit_status}</span><br>
            類別：{cat}<br>
            <a href="{link}" target="_blank">Google 導航 ➜</a></div>
            """

            folium.Marker(
                location=[lat, lng],
                popup=folium.Popup(popup_html, max_width=250),
                tooltip=name,
                icon=folium.Icon(**icon_arg)
            ).add_to(marker_cluster)

        folium.LayerControl().add_to(m)
        map_html = m._repr_html_()

        return map_html, df, f"{debug_msg} (地圖顯示 {len(df)} 筆)"

    except Exception as e:
        return None, pd.DataFrame(), f"❌ 程式崩潰: {str(e)}"

def ask_map_ai_logic(df_data):
    """
    地圖 AI (v4)：根據資料來源變更更新指示詞
    """
    if df_data is None or df_data.empty:
        return "⚠️ 請先點擊「更新地圖」來取得地點資料，才能進行分析。"

    try:
        col_name = '名稱'
        col_category = '類別'

        # 隨機選取 3 個地點
        sample_df = df_data.sample(n=min(3, len(df_data)))

        # 構建地點資訊字串
        places_info = []
        for _, r in sample_df.iterrows():
            # 讀取剛剛在地圖邏輯產生的 'is_visited' 欄位
            visited_mark = "【已去過】" if r.get('is_visited', False) else "【未去過】"
            places_info.append(f"{r[col_name]} ({r[col_category]}) - {visited_mark}")

        places_text = "、".join(places_info)

        # 更新 Prompt
        prompt = f"""
        使用者正在查看運動/休閒地圖，這些地點資料來自使用者的運動地圖資料庫。
        目前的篩選清單中有這幾個地點：{places_text}。

        請扮演一位熱情的健身導遊，針對這 3 個地點給出推薦建議。

        請根據地點後面的【已去過】或【未去過】標籤來調整語氣：
        1. 對於【已去過】的地點：可以問候使用者是否想念那裡的環境，或者建議進行不同強度的訓練。
        2. 對於【未去過】的地點：大力推薦它的特色，鼓勵使用者去探索新環境。

        最後，請給我一個「本日最佳選擇」並說明理由。
        請使用繁體中文，直接開始介紹。
        """

        response = GEMINI_MODEL.generate_content(prompt)

        if hasattr(response, 'text') and response.text:
            ai_advice = response.text
            ai_advice = ai_advice.replace("*", "").replace("#", "")
            return ai_advice
        else:
            return "⚠️ AI 已讀取但未回傳內容。"

    except Exception as e:
        return f"❌ AI 分析失敗：{str(e)}"

In [12]:
def ask_character_ai_core(char_key, data_summary):
    """
    仿照地圖 AI 寫法：專門處理角色對話生成
    """
    if not data_summary:
        return "⚠️ 指導者目前沒有足夠的資訊可以進行評估。"

    # 取得對應角色的資訊
    character_info = CHARACTERS.get(char_key)
    if not character_info:
        return f"❌ 找不到角色資料: {char_key}"

    # 構建結構化的 Prompt
    prompt = f"""
    你現在的角色是：【{char_key}】
    你的個性設定：{character_info['personality']}

    以下是使用者的「今日運動目標與狀態」摘要：
    {data_summary}

    請針對以上數據，以你的角色口吻給予一段點評與建議：
    1. 點評：針對使用者完成目標的情況（或尚未完成）給予回應。
    2. 指導：給出一項關於「飲食」或「訓練」的具體建議。
    3. 結語：用符合你人設的方式結束對話（例如霸道總裁的命令、奶狗的撒嬌）。

    請使用繁體中文，善用 Markdown 標題（例如 ###），直接開始回覆。
    """

    try:
        # 使用全域的 model 物件進行生成
        response = GEMINI_MODEL.generate_content(prompt)
        if hasattr(response, 'text') and response.text:
            return response.text.strip()
        return "⚠️ 指導者已讀取資料，但陷入了沉思（未回傳內容）。"
    except Exception as e:
        return f"❌ AI 互動失敗：{str(e)}"

def generate_gallery_html():
    html = """<style>.gallery-container { display: grid; grid-template-columns: repeat(auto-fill, minmax(250px, 1fr)); gap: 15px; padding: 10px; max-height: 500px; overflow-y: auto; } .gallery-item { border: 2px solid #ddd; border-radius: 10px; padding: 15px; background-color: #f9f9f9; position: relative; min-height: 100px; display: flex; flex-direction: column; justify-content: center; align-items: center; text-align: center; } .gallery-item.collected { border-color: #ff9800; background-color: #fff8e1; } .stamp { position: absolute; top: 5px; right: 5px; width: 60px; height: 60px; border: 3px solid #d32f2f; border-radius: 50%; color: #d32f2f; font-weight: bold; font-size: 14px; display: flex; align-items: center; justify-content: center; transform: rotate(-20deg); opacity: 0.8; background: rgba(255, 255, 255, 0.8); z-index: 10; } .char-tag { font-size: 12px; color: #666; margin-bottom: 5px; font-weight: bold; } .revealed-text { font-size: 14px; color: #333; } </style><div class="gallery-container">"""
    for item in ALL_REWARDS_FLAT:
        is_collected = item["text"] in player_stats["collected_items"]
        if is_collected:
            card_html = f"""<div class="gallery-item collected"><div class="stamp">已收集</div><div class="char-tag">【{item['character']}】</div><div class="revealed-text">{item['text']}</div></div>"""
        else:
            card_html = f"""<div class="gallery-item"><div class="char-tag">【{item['character']}】</div><div style="font-size: 24px; color: #aaa;">???</div><div style="font-size: 12px; color: #ccc;">尚未解鎖</div></div>"""
        html += card_html
    html += "</div>"
    return html

def get_diet_suggestion(character_name, current_goals_df):
    """
    修正版：接收兩個參數，並將目標清單轉化為文字摘要
    """
    # 角色名稱轉換邏輯
    char_key = character_name
    if "溫柔" in character_name: char_key = "溫柔可靠男"
    elif "禁慾" in character_name: char_key = "禁慾腹黑男"
    elif "危險" in character_name: char_key = "危險男友"
    elif "奶狗" in character_name: char_key = "奶狗"
    elif "霸道" in character_name: char_key = "霸道總裁"

    # 1. 準備數據摘要 (將 Dataframe 轉為文字)
    goal_summary = []
    if current_goals_df is not None and not current_goals_df.empty:
        for _, row in current_goals_df.iterrows():
            status = "【✅ 已達成】" if row['是否完成'] else "【❌ 尚未完成】"
            goal_summary.append(f"- {row['目標項目']}: {status}")

    data_summary_text = "\n".join(goal_summary) if goal_summary else "目前尚未設定任何目標。"

    # 加入心動值作為 AI 參考
    data_summary_text += f"\n- 目前心動值: {player_stats['heart_value']}"

    # 2. 呼叫核心函式取得回應
    return ask_character_ai_core(char_key, data_summary_text)

def get_gacha_reward(current_character_name):
    if player_stats["is_punished"]:
        return ("⚠️ 系統錯誤：你今天沒有完成目標，已被指導者拉黑，無法進行互動。", player_stats["heart_value"], player_stats["plot_unlocked"], player_stats["gacha_count"], IMAGE_PATHS["default_scene"], generate_gallery_html(), gr.update(), gr.update(), "")

    char_key = current_character_name
    if "溫柔" in current_character_name: char_key = "溫柔可靠男"
    elif "禁慾" in current_character_name: char_key = "禁慾腹黑男"
    elif "危險" in current_character_name: char_key = "危險男友"
    elif "奶狗" in current_character_name: char_key = "奶狗"

    if char_key in REWARDS_BY_CHARACTER: reward_pool = REWARDS_BY_CHARACTER[char_key]
    else: reward_pool = [("系統忙線中，請重新嘗試", 0, "default_scene")]

    selected_reward_tuple = random.choice(reward_pool)
    selected_reward_text = selected_reward_tuple[0]
    heart_bonus = selected_reward_tuple[1]
    image_key = selected_reward_tuple[2]

    player_stats["collected_items"].add(selected_reward_text)
    player_stats["plot_unlocked"] = len(player_stats["collected_items"])

    is_secret_visible = False
    is_floating_btn_visible = False
    secret_story_content = player_stats["current_secret_content"]

    # 檢查隱藏劇情
    all_rewards_for_char = [r[0] for r in REWARDS_BY_CHARACTER[char_key]]
    collected_for_char = [txt for txt in all_rewards_for_char if txt in player_stats["collected_items"]]
    if len(collected_for_char) == len(all_rewards_for_char):
        if char_key not in player_stats["unlocked_secrets"]:
            player_stats["unlocked_secrets"].add(char_key)
            secret_story_content = SECRET_STORIES.get(char_key, "恭喜解鎖隱藏劇情！")
            player_stats["current_secret_content"] = secret_story_content
            is_secret_visible = True
            is_floating_btn_visible = False
            gr.Info(f"🎉 恭喜！解鎖了 {char_key} 的隱藏告白！")
        else:
             if player_stats["current_secret_content"]: is_floating_btn_visible = True

    display_image = IMAGE_PATHS.get(image_key, IMAGE_PATHS["default_scene"])
    player_stats["heart_value"] += heart_bonus
    player_stats["gacha_count"] += 1
    new_gallery_html = generate_gallery_html()

    return (f"🎉 {char_key} 的專屬劇情：\n{selected_reward_text}", player_stats["heart_value"], player_stats["plot_unlocked"], player_stats["gacha_count"], display_image, new_gallery_html, gr.update(visible=is_secret_visible), gr.update(visible=is_floating_btn_visible), secret_story_content)

def update_character_display(character_choice):
    player_stats["selected_character"] = character_choice
    char_key = character_choice
    if "溫柔" in character_choice: char_key = "溫柔可靠男"
    elif "禁慾" in character_choice: char_key = "禁慾腹黑男"
    elif "危險" in character_choice: char_key = "危險男友"
    elif "奶狗" in character_choice: char_key = "奶狗"
    char_info = CHARACTERS[char_key]
    return char_info["images"]["normal"], f"目前指導者: {char_key}"

# --- NEW: 目標管理系統相關函式 ---

def add_goal(text, current_df):
    """新增目標到表格"""
    if not text:
        return current_df, "" # 不做任何事
    new_data = pd.DataFrame({"目標項目": [text], "是否完成": [False]})
    updated_df = pd.concat([current_df, new_data], ignore_index=True)
    return updated_df, "" # 更新表格並清空輸入框

def check_goals_completion(df):
    """
    檢查表格狀態，決定是否解鎖「是！我完成了」按鈕
    規則：
    1. 表格不能是空的 (至少要有一個目標)
    2. 表格中所有「是否完成」必須為 True
    """
    if df is None or df.empty:
        return gr.update(interactive=False, value="⏳ 請先新增今日目標")

    # 檢查是否所有 checkbox 都是 True
    # Gradio dataframe 的 checkbox欄位會回傳 boolean
    all_done = df["是否完成"].all()

    if all_done:
        return gr.update(interactive=True, value="✅ 是！我完成了")
    else:
        return gr.update(interactive=False, value="🔒 尚未全部完成")

def clear_all_goals():
    """清空所有目標"""
    return pd.DataFrame(columns=["目標項目", "是否完成"])

def export_goals_csv(df):
    """將今日目標表格下載為 CSV"""
    file_path = "/content/today_goals.csv"
    df.to_csv(file_path, index=False)
    return file_path

def close_overlays():
    """關閉達成或失敗的覆蓋層"""
    return gr.update(visible=False), gr.update(visible=False)

def close_secret_letter():
    """關閉隱藏信件視窗"""
    return gr.update(visible=False), gr.update(visible=True)

def open_secret_letter():
    """開啟隱藏信件視窗"""
    return gr.update(visible=True), gr.update(visible=False)

def handle_goal_yes():
    player_stats["is_punished"] = False
    return (gr.update(visible=True), gr.update(visible=False), gr.update(interactive=True, value="🎁 觸發專屬劇情 (互動)"))

def handle_goal_no():
    player_stats["is_punished"] = True
    return (gr.update(visible=False), gr.update(visible=True), gr.update(interactive=False, value="🚫 權限被鎖定 (未達成目標)"))

In [13]:
interaction_css = """
.gradio-container { background-color: #f0f2f5; transition: background-color 0.8s ease, filter 0.8s ease; }
.global-punishment { background-color: #000000 !important; filter: grayscale(90%) contrast(120%) !important; }
.global-punishment .gradio-container { background-color: #000000 !important; }
.global-punishment p, .global-punishment h1, .global-punishment span { color: #cc0000 !important; text-shadow: 0 0 5px #330000; }
.secret-modal { position: fixed !important; top: 50% !important; left: 50% !important; transform: translate(-50%, -50%) !important; width: 80% !important; max-width: 600px !important; z-index: 9999 !important; background-color: #FFF8DC !important; border: 4px double #FFD700 !important; border-radius: 15px !important; padding: 20px !important; box-shadow: 0 10px 25px rgba(0,0,0,0.5) !important; }
.floating-envelope { position: fixed !important; bottom: 30px !important; right: 30px !important; width: 60px !important; height: 60px !important; border-radius: 50% !important; background: linear-gradient(135deg, #ff9a9e 0%, #fecfef 99%, #fecfef 100%) !important; box-shadow: 0 4px 10px rgba(0,0,0,0.3) !important; z-index: 9998 !important; font-size: 30px !important; border: 2px solid white !important; }
.overlay-container { position: fixed !important; inset: 0 !important; width: 100vw !important; height: 100vh !important; z-index: 999999 !important; display: flex !important; flex-direction: column !important; justify-content: center !important; align-items: center !important; padding: 20px !important; text-align: center !important; }
.celebration-mode { background-color: rgba(255, 255, 255, 0.98) !important; color: #333 !important; }
.boss-angry-mode { background-color: rgba(0, 0, 0, 0.95) !important; color: #ff3333 !important; font-family: 'Courier New', monospace !important; border: 1px solid #330000; }
.overlay-title { font-size: 40px !important; font-weight: bold !important; margin-bottom: 20px !important; }
.overlay-text { font-size: 20px !important; margin-bottom: 40px !important; max-width: 600px !important; }
.overlay-btn { font-size: 20px !important; padding: 10px 30px !important; }
.close-btn { position: absolute !important; top: 10px !important; right: 10px !important; background: transparent !important; border: none !important; font-size: 20px !important; cursor: pointer !important; color: #8b4500 !important; }
"""

js_turn_dark = """function() { document.body.classList.add('global-punishment'); document.querySelector('.gradio-container').classList.add('global-punishment'); }"""
js_turn_light = """function() { document.body.classList.remove('global-punishment'); document.querySelector('.gradio-container').classList.remove('global-punishment'); }"""


In [16]:
# ---------------------------------------------------------
# Gradio 介面 (UI Construction)
# ---------------------------------------------------------

# 讀取初始值
initial_user_records_full = get_user_records_for_dropdown()
initial_user_names = get_user_names_for_current_dropdown()
initial_selected_user = initial_user_names[0] if initial_user_names else None
initial_records = get_recent_records(initial_selected_user)

# 這裡的值需要判斷，如果沒有有效紀錄，就不要選中任何值
initial_record_value = initial_records[0] if initial_records and "尚未有訓練紀錄" not in initial_records[0] and initial_records[0] != "請先選擇使用者" else None


with gr.Blocks(title="【戀愛契約】霸道總裁的私人特訓", css=interaction_css) as demo:
    # 覆蓋層 (保持不變)
    with gr.Group(visible=False, elem_classes=["overlay-container", "celebration-mode"]) as overlay_happy:
        gr.Markdown("# 🎉 太棒了！達成目標！", elem_classes="overlay-title")
        gr.Markdown("你真棒！為了獎勵你的自律，**系統已解鎖今日的「親密互動」權限**。\n快去和你的指導者領取獎勵吧！", elem_classes="overlay-text")
        btn_close_happy = gr.Button("💖 馬上去互動", elem_classes="overlay-btn")

    with gr.Group(visible=False, elem_classes=["overlay-container", "boss-angry-mode"]) as overlay_angry:
        gr.Markdown("# 😡 霸總震怒模式", elem_classes="overlay-title")
        gr.Markdown("### 「連這一點小事都做不到嗎？」\n\n黑暗中傳來他冰冷的聲音：「既然你沒有完成今天的目標，就別想從我這裡得到任何甜頭。」\n\n**懲罰執行中：**\n1. 今日所有「親密互動」功能已被鎖定。\n2. 你只能使用「飲食建議」，好好反省你的菜單。\n3. 明天不准再失敗。", elem_classes="overlay-text")
        btn_close_angry = gr.Button("😞 我知道了...去反省", elem_classes="overlay-btn")
    gr.Markdown("# 🏋️‍♂️ 【戀愛契約】霸道總裁的私人特訓 ")

    with gr.Tabs():
        # --- TAB 1: 每日訓練紀錄 (拆分後的母 Tab) ---
        with gr.TabItem("📝 每日訓練紀錄"):
            # 將下拉選單放在 Tab 1 頂部，作為該 Tab 的控制項
            current_user_dropdown = gr.Dropdown(
                label="目前選擇的使用者",
                choices=initial_user_names,
                value=initial_selected_user,
                interactive=True
            )

            # 新增子 Tabs
            with gr.Tabs():

                # --- 子 Tab 1.1: 基本資料與管理 (左輸入/右操作) ---
                with gr.TabItem("👤 基本資料與管理"):
                    gr.Markdown("## 1. 初始化/更新 基本資料")
                    with gr.Row():
                        # 左側 Column 1 (輸入欄位)
                        with gr.Column(scale=1):
                            gr.Markdown("### 📝 輸入/更新個人資料")
                            name = gr.Textbox(label="姓名")
                            gender = gr.Dropdown(["男", "女", "其他"], label="性別")
                            height = gr.Number(label="身高 (cm)")
                            weight = gr.Number(label="體重 (kg)")
                            schedule = gr.Textbox(label="時間表", placeholder="例如: 一三五重訓")
                            goal = gr.Textbox(label="目標", placeholder="例如: 增肌減脂")

                        # 右側 Column 2 (操作與刪除區)
                        with gr.Column(scale=1):
                            gr.Markdown("### ⚙️ 使用者資料管理")

                            # 刪除使用者資料的下拉選單
                            user_record_to_del = gr.Dropdown(
                                label="選擇要刪除的使用者紀錄",
                                choices=initial_user_records_full,
                                multiselect=False,
                                value=initial_user_records_full[0] if initial_user_records_full else None
                            )

                            # 💡 按鈕區塊：將按鈕和訊息欄位放在右側 Column 內
                            with gr.Group():
                                with gr.Row(equal_height=True):
                                    btn_save_user = gr.Button("💾 儲存/更新使用者資料", scale=1)
                                    btn_delete_selected_user = gr.Button("🗑️ 刪除選定使用者資料", variant="secondary", scale=1)

                            # 統一的訊息欄位：放在右側 Column 的最下方
                            gr.Markdown("--- \n#### 操作訊息")
                            unified_user_op_msg = gr.Label(label="使用者管理操作結果")

                # --- 子 Tab 1.2: 運動紀錄操作 (新增/刪除操作及訊息全都在右側) ---
                with gr.TabItem("💪 運動紀錄操作"):
                    gr.Markdown("## 2. 新增 / 刪除 運動紀錄")

                    # 💡 修正: 使用 gr.Row 將輸入欄位(左)和所有操作(右)並列
                    with gr.Row(equal_height=True):
                        # 左側 Column：新增紀錄 (只有輸入欄位)
                        with gr.Column(scale=1):
                            gr.Markdown("### 📝 輸入新的訓練紀錄")
                            exercise = gr.Textbox(label="運動項目", placeholder="例如: 深蹲")
                            sets = gr.Number(label="組數", value=3)
                            reps = gr.Textbox(label="次數 / 時間", placeholder="例如: 12下 or 30分")
                            weight_val = gr.Textbox(label="重量 / 公里數", placeholder="例如: 60kg or 5km")
                            user_feeling = gr.Textbox(label="今日體感", placeholder="例如：腿好痠")

                        # 右側 Column：刪除操作、新增按鈕、刪除按鈕、訊息欄 (全部都在右側)
                        with gr.Column(scale=1):
                            gr.Markdown("### ⚙️ 紀錄操作")

                            # 1. 刪除選單
                            record_to_del = gr.Dropdown(
                                label="選擇要刪除的最近 5 筆運動紀錄",
                                choices=initial_records,
                                multiselect=False,
                                value=initial_record_value
                            )
                            # 2. 按鈕並列 (使用 Group 和 Row)
                            with gr.Group():
                                with gr.Row(equal_height=True):
                                    # 注意：這裡的 btn_add_record 雖然在右側，但會操作左側的輸入值
                                    btn_add_record = gr.Button("➕ 新增運動紀錄", scale=1)
                                    btn_del_record = gr.Button("🗑️ 刪除選定資料", variant="secondary", scale=1)

                            # 3. 訊息欄位
                            gr.Markdown("--- \n#### 紀錄操作訊息")
                            record_op_msg = gr.Label(label="運動紀錄操作結果")

                            # 4. 占位符 (確保右側 Column 高度與左側 5 個輸入欄位對齊)
                            # 左側有 5 個輸入欄位。右側目前有 1個Dropdown + 2個按鈕 + 1個Label。
                            # 為了確保高度對齊，我們需要 5 個 Placeholder 來填補空間。
                            # 由於 Dropdown, Group (2按鈕) 和 Label 的高度各不相同，
                            # 最保險的做法是讓右側的元素盡量與左側的輸入欄位數量匹配
                            # 這裡使用 4 個隱藏 Textbox 來調整高度，確保與左側輸入欄位對齊
                            gr.Textbox(label="", visible=False)
                            gr.Textbox(label="", visible=False)
                            gr.Textbox(label="", visible=False)
                            gr.Textbox(label="", visible=False)
                            gr.Textbox(label="", visible=False)

        # --- TAB 2: 訓練週報產生器 ---
        with gr.TabItem("📊 訓練週報產生器"):
            gr.Markdown("## 👤 選擇要生成週報的使用者")
            # 將下拉選單放在 Tab 2 頂部，作為該 Tab 的控制項
            current_user_dropdown_tab2 = gr.Dropdown(
                label="目前選擇的使用者",
                choices=initial_user_names,
                value=initial_selected_user,
                interactive=True
            )

            gr.Markdown("---\n系統將讀取近 7 天資料，計算總量並進行 AI 分析。**API Key 已自動從 Colab Secret 讀取。**")
            btn_gen_report = gr.Button("🚀 生成本週分析週報")

            with gr.Row():
                with gr.Column(scale=1):
                    stats_output = gr.Textbox(label="統計摘要")

                    gr.Markdown("### 👔 AI 週報建議")
                    report_output = gr.Markdown(label="Gemini AI 教練週報")
                with gr.Column(scale=1):
                    # 圖表 1：訓練總量 (垂直)
                    plot_vol = gr.BarPlot(label="📈 每週訓練總量", x="週數", y="訓練總量", color="訓練總量", height=300)

                    # 💡 新增圖表 2：訓練天數 (垂直)
                    plot_days = gr.BarPlot(label="📅 每週訓練天數", x="週數", y="天數", color="天數", height=300)

                    # 💡 新增圖表 3：體感熱詞 (水平，像 PTT 那樣)
                    plot_hot = gr.BarPlot(label="🔥 訓練體感熱詞 Top 10", x="出現次數", y="詞彙", color="出現次數", height=300)

        # --- TAB 3: 歷史資料搜尋與匯出 ---
        with gr.TabItem("📜 歷史資料搜尋與匯出"):
            gr.Markdown("### 🔍 進階歷史搜尋")
            gr.Markdown("系統會自動載入**目前選擇使用者**的歷史資料。")

            with gr.Row():
                with gr.Column(scale=1):
                    # 1. 日期篩選：下拉選單
                    history_date_filter = gr.Dropdown(
                        label="日期範圍",
                        choices=["全部", "近一周", "近一個月", "近三個月"],
                        value="全部",
                        interactive=True
                    )

                with gr.Column(scale=1):
                    # 2. 運動項目篩選：下拉選單 (會自動更新)
                    history_exercise_filter = gr.Dropdown(
                        label="運動項目",
                        choices=["全部"], # 初始值，會被程式自動更新
                        value="全部",
                        interactive=True,
                        allow_custom_value=False # 強制只能選清單內的
                    )

                with gr.Column(scale=1):
                    gr.Markdown(" ") # 佔位符，讓按鈕往下對齊
                    history_search_btn = gr.Button("🔎 開始搜尋", variant="primary")

            history_msg = gr.Label(label="搜尋狀態")

            gr.Markdown("### 📊 搜尋結果")
            history_results_table = gr.Dataframe(
                label="符合條件的紀錄",
                interactive=False,
                wrap=True
            )

            gr.Markdown("---")
            gr.Markdown("### 📤 匯出資料")
            with gr.Row():
                with gr.Column():
                    history_export_format = gr.Radio(["CSV", "JSON"], label="選擇匯出格式", value="CSV")
                    history_export_btn = gr.DownloadButton("📥 匯出結果", variant="secondary")

        # --- TAB 4: 運動地圖 ---
        with gr.TabItem("🗺️ 運動地圖"):
            gr.Markdown("## 🗺️ 智慧地圖：地點檢視 + AI 導遊")

            current_map_df_state = gr.State(pd.DataFrame())
            # 重新取得地點清單，確保下拉選單有資料
            all_locations = get_all_map_locations()

            # 預設類別 (若讀取失敗時的備案)
            map_cats_choices = ["健身房", "公園"]

            # 【上半部】控制面板區域 (水平並列)
            with gr.Row():

                # --- 左側區塊：地圖篩選與更新 ---
                with gr.Column(scale=1):
                    gr.Markdown("### 🛠️ 1. 地圖篩選設定")
                    with gr.Row():
                        chk_today = gr.Checkbox(label="僅顯示今日營業", value=False)
                        filter_visit_status = gr.Radio(
                            label="造訪狀態",
                            choices=["全部", "已去過", "未去過"],
                            value="全部",
                            interactive=True
                        )
                    chk_cats = gr.CheckboxGroup(label="選擇類別", choices=map_cats_choices, value=map_cats_choices)

                    with gr.Row():
                        btn_map = gr.Button("📍 更新地圖", variant="primary")

                    txt_status = gr.Textbox(label="系統狀態", interactive=False, lines=1)

                # --- 右側區塊：打卡操作 & AI 分析 ---
                with gr.Column(scale=1):
                    gr.Markdown("### 🚩 2. 地點打卡 (手動記錄)")
                    with gr.Row():
                        dd_locations = gr.Dropdown(label="選擇地點", choices=all_locations, interactive=True, scale=2)
                        with gr.Column(scale=1, min_width=120):
                            btn_visit_yes = gr.Button("✅ 標記去過", size="sm")
                            btn_visit_no = gr.Button("❌ 標記沒去", size="sm", variant="secondary")

                    gr.Markdown("---")
                    gr.Markdown("### 🤖 3. AI 導遊建議")
                    btn_map_ai = gr.Button("✨ 請 AI 分析目前的篩選結果", variant="secondary")
                    # 高度設小一點 (lines=5)，以免佔據太多地圖空間
                    txt_map_ai_output = gr.Textbox(label="Gemini 建議", lines=5, interactive=False)

            # 【下半部】地圖顯示區域 (放最下面，寬度全開)
            gr.Markdown("---")
            gr.Markdown("### 🗺️ 地圖瀏覽")
            # 設定 min_height 讓地圖在下方夠大
            html_map = gr.HTML(label="地圖", min_height=600)


        # --- TAB 5: 💖 指導者互動系統 ---
        with gr.TabItem("💖 指導者互動系統"):
            gr.Markdown("# 💘 戀與健身中心：解鎖你的專屬劇情")

            with gr.Tabs():
                # --- 子分頁：今日任務設定 ---
                with gr.Tab("📝 設定今日目標"):
                    gr.Markdown("### 🎯 任務清單 (完成後可解鎖互動權限)")
                    with gr.Row():
                        goal_input = gr.Textbox(label="輸入運動項目", placeholder="例如: 深蹲 20 下", scale=4)
                        btn_add_goal = gr.Button("➕ 新增目標", scale=1, variant="primary")

                    goals_df = gr.Dataframe(
                        headers=["目標項目", "是否完成"],
                        datatype=["str", "bool"],
                        col_count=(2, "fixed"),
                        interactive=True,
                        value=[["熱身 10 分鐘", False]],
                        label="今日任務清單 (請直接在表格打勾)"
                    )

                    with gr.Row():
                        btn_download_csv = gr.DownloadButton("📥 下載 CSV", label="下載清單")
                        btn_clear_goals = gr.Button("🗑️ 清空列表", variant="stop")

                    gr.Markdown("---")
                    gr.Markdown("### 📅 任務結算")
                    with gr.Row():
                        # 初始狀態為鎖定，需完成所有目標才解鎖
                        btn_goal_yes = gr.Button("⏳ 請先完成上方所有目標", interactive=False, variant="primary")
                        btn_goal_no = gr.Button("❌ 沒有...我偷懶了", variant="stop")

                # --- 子分頁：角色互動與劇情 ---
                with gr.Tab("💬 指導者互動"):
                    with gr.Row():
                        # 左側：角色切換與功能按鈕
                        with gr.Column(scale=1):
                            character_selector = gr.Radio(
                                choices=["霸道總裁", "奶狗", "危險男友", "禁慾腹黑男", "溫柔可靠男"],
                                label="選擇你的指導者",
                                value="霸道總裁",
                                interactive=True
                            )
                            current_character_image = gr.Image(
                                value=LOCAL_IMAGES["boss_normal"],
                                label="目前指導者",
                                height=400, width=300,
                                interactive=False
                            )
                            character_name_display = gr.Textbox(value="目前角色: 霸道總裁", label="角色狀態", interactive=False)

                            diet_button = gr.Button("🍱 詢問今日飲食建議")
                            gacha_button = gr.Button("🎁 觸發專屬劇情 (互動)", interactive=False) # 預設鎖定

                        # 右側：劇情回饋與圖鑑
                        with gr.Column(scale=2):
                            # 隱藏信件按鈕 (解鎖後出現)
                            btn_floating_mail = gr.Button("💌", elem_classes="floating-envelope", visible=False)

                            # 隱藏劇情彈窗邏輯
                            with gr.Group(visible=False, elem_classes="secret-modal") as secret_group:
                                with gr.Row():
                                    gr.Markdown("## ✨ 隱藏信件已開啟 ✨")
                                    btn_close_secret = gr.Button("✖️", elem_classes="close-btn", scale=0)
                                secret_story_display = gr.Markdown(value="...", elem_classes="secret-text")

                            with gr.Tabs():
                                with gr.Tab("對話與劇情"):
                                    game_output = gr.Textbox(label="指導者回應 / 劇情結果", lines=6, interactive=False)
                                    scene_image = gr.Image(value="https://dummyimage.com/600x400/ccc/000&text=Gym+Scene", label="互動場景", height=400, interactive=False)

                                with gr.Tab("📖 劇情圖鑑"):
                                    gr.Markdown("### 🌟 你的甜蜜回憶錄")
                                    gallery_display = gr.HTML(value=generate_gallery_html())

                                with gr.Tab("💓 親密度數據"):
                                    heart_bar = gr.Slider(label="❤️ 心動值", minimum=0, maximum=100, value=0, interactive=False)
                                    with gr.Row():
                                        plot_count = gr.Number(label="已解鎖劇情總數", value=0, interactive=False)
                                        gacha_count_display = gr.Number(label="總互動次數", value=0, interactive=False)

    # =========================================================
    # 6. 函數綁定
    # =========================================================

    # 綁定：使用者管理
    btn_save_user.click(
        save_user_profile,
        inputs=[name, gender, height, weight, schedule, goal],
        outputs=[unified_user_op_msg, user_record_to_del, current_user_dropdown, current_user_dropdown_tab2, record_to_del]
    )

    btn_delete_selected_user.click(
        delete_selected_user_record,
        inputs=[user_record_to_del],
        outputs=[unified_user_op_msg, user_record_to_del, current_user_dropdown, current_user_dropdown_tab2, record_to_del]
    )

    # 綁定：運動紀錄操作 (輸出仍指向 record_op_msg)
    btn_add_record.click(
        add_exercise_record,
        inputs=[current_user_dropdown, exercise, sets, reps, weight_val, user_feeling],
        outputs=[record_op_msg, record_to_del]
    )
    btn_del_record.click(
        delete_record,
        inputs=[current_user_dropdown, record_to_del],
        outputs=[record_op_msg, record_to_del]
    )

    # 綁定：訓練週報
    btn_gen_report.click(
        generate_weekly_report_logic,
        inputs=[current_user_dropdown_tab2],
        outputs=[
            report_output, # 對應 final_report_md
            plot_vol,      # 對應 weekly_vol_df
            stats_output,  # 對應 stats_text
            plot_hot,      # 對應 hot_words_df (熱詞圖)
            plot_days      # 對應 weekly_days_df (天數圖)
        ]
    )

    def on_user_change(user_name):
        # 1. 刷新紀錄刪除選單 (原本功能)
        rec_update = refresh_recent_records(user_name)
        # 2. 載入歷史資料並取得運動項目列表 (新功能)
        ex_filter_update = load_and_update_history_filters(user_name)
        # 3. 回傳給各個元件
        # 順序: record_to_del (Tab1), current_user_dropdown_tab2 (Tab2), history_exercise_filter (Tab3)
        return rec_update, user_name, ex_filter_update

    def on_user_change_tab2(user_name):
        rec_update = refresh_recent_records(user_name)
        ex_filter_update = load_and_update_history_filters(user_name)
        return rec_update, user_name, ex_filter_update

    # 定義一個「超級組合函式」，處理切換使用者時的所有連動
    def on_user_switch_combined(filter_today, selected_cats, visit_status, user_name):
        # 1. 執行地圖更新 (Tab 4)
        map_html, map_state, map_msg = update_map_logic(filter_today, selected_cats, visit_status, user_name)

        # 2. 執行歷史資料選單更新 (Tab 3)
        history_filter_update = load_and_update_history_filters(user_name)

        # 3. 執行近期紀錄更新 (Tab 1)
        record_del_update = refresh_recent_records(user_name)

        # 回傳順序必須對應下方的 outputs
        # user_name 是為了同步另一個分頁的下拉選單
        return record_del_update, user_name, history_filter_update, map_html, map_state, map_msg

    # 1. 綁定 Tab 1 下拉選單的改變事件
    current_user_dropdown.change(
        fn=on_user_switch_combined,
        inputs=[chk_today, chk_cats, filter_visit_status, current_user_dropdown],
        outputs=[
            record_to_del,               # Tab 1: 刪除紀錄選單
            current_user_dropdown_tab2,  # Tab 2: 同步選單
            history_exercise_filter,     # Tab 3: 運動項目選單 [這裡就會更新了!]
            html_map,                    # Tab 4: 地圖
            current_map_df_state,        # Tab 4: 地圖資料
            txt_status                   # Tab 4: 系統狀態
        ]
    )

    # 2. 綁定 Tab 2 下拉選單的改變事件 (反向同步)
    current_user_dropdown_tab2.change(
        fn=on_user_switch_combined,
        inputs=[chk_today, chk_cats, filter_visit_status, current_user_dropdown_tab2],
        outputs=[
            record_to_del,               # Tab 1: 刪除紀錄選單
            current_user_dropdown,       # Tab 1: 同步選單 (反向)
            history_exercise_filter,     # Tab 3: 運動項目選單
            html_map,                    # Tab 4: 地圖
            current_map_df_state,        # Tab 4: 地圖資料
            txt_status                   # Tab 4: 系統狀態
        ]
    )

    # 綁定 Tab 3 的搜尋按鈕
    history_search_btn.click(
        fn=search_history_logic,
        inputs=[history_date_filter, history_exercise_filter],
        outputs=[history_results_table, history_msg]
    )

    # 綁定 Tab 3 的匯出按鈕
    history_export_btn.click(
        fn=export_filtered_history,
        inputs=[history_export_format],
        outputs=[history_export_btn]
    )

    # 按鈕：更新地圖
    btn_map.click(
        fn=update_map_logic,  # 呼叫我們在第三步加的函式
        inputs=[chk_today, chk_cats, filter_visit_status, current_user_dropdown],
        outputs=[html_map, current_map_df_state, txt_status] # 注意：這裡同時更新了 State
    )

    # 手動打卡按鈕 (標記為已去過)
    btn_visit_yes.click(
        fn=toggle_visit_status,
        inputs=[current_user_dropdown, dd_locations, gr.State("標記為已去過")], # 傳入 State 作為固定參數
        outputs=[txt_status, html_map] # 打卡完自動更新狀態和地圖
    ).then( # 串聯事件：打卡完後，重新執行一次地圖更新邏輯，確保畫面變色
        fn=update_map_logic,
        inputs=[chk_today, chk_cats, filter_visit_status, current_user_dropdown],
        outputs=[html_map, current_map_df_state, txt_status]
    )

    # 取消打卡按鈕 (標記為未去過)
    btn_visit_no.click(
        fn=toggle_visit_status,
        inputs=[current_user_dropdown, dd_locations, gr.State("標記為未去過")],
        outputs=[txt_status, html_map]
    ).then(
        fn=update_map_logic,
        inputs=[chk_today, chk_cats, filter_visit_status, current_user_dropdown],
        outputs=[html_map, current_map_df_state, txt_status]
    )

    # 按鈕：AI 分析
    btn_map_ai.click(
        fn=ask_map_ai_logic,  # 呼叫我們在第三步加的函式
        inputs=[current_map_df_state], # 輸入來源是那個看不見的 State
        outputs=[txt_map_ai_output]
    )

    def on_app_load(filter_today, selected_cats, visit_status, user_name):
        """
        當程式啟動時，一次執行所有必要的載入動作：
        1. 繪製地圖 (Tab 4)
        2. 讀取該使用者的運動歷史，更新運動項目選單 (Tab 3)
        3. 刷新刪除紀錄的選單 (Tab 1)
        """
        # 1. 執行地圖邏輯
        map_html, map_state, map_msg = update_map_logic(filter_today, selected_cats, visit_status, user_name)

        # 2. 執行歷史資料讀取 (這就是之前漏掉的部分！)
        history_filter_update = load_and_update_history_filters(user_name)

        # 3. 執行紀錄刷新
        record_del_update = refresh_recent_records(user_name)

        # 回傳所有結果
        return map_html, map_state, map_msg, history_filter_update, record_del_update

    # 綁定啟動事件
    demo.load(
        fn=on_app_load,
        inputs=[chk_today, chk_cats, filter_visit_status, current_user_dropdown],
        outputs=[
            html_map,                # 更新地圖
            current_map_df_state,    # 更新地圖 State
            txt_status,              # 更新地圖狀態文字
            history_exercise_filter, # ✅ 更新 Tab 3 的運動項目選單
            record_to_del            # ✅ 更新 Tab 1 的刪除紀錄選單
        ]
    )

    # --- 事件綁定 ---

    # 0. 角色切換自動更新圖片與狀態文字
    character_selector.change(
        update_character_display,
        inputs=character_selector,
        outputs=[current_character_image, character_name_display]
    )

    # 1. 新增目標邏輯
    btn_add_goal.click(
        add_goal,
        inputs=[goal_input, goals_df],
        outputs=[goals_df, goal_input]
    ).then( # 新增完後馬上檢查一次狀態
        check_goals_completion,
        inputs=[goals_df],
        outputs=[btn_goal_yes]
    )

    # 2. 表格更動邏輯 (打勾/取消打勾/編輯) -> 連動檢查按鈕
    goals_df.change(
        check_goals_completion,
        inputs=[goals_df],
        outputs=[btn_goal_yes]
    )

    # 3. 清空與下載
    btn_clear_goals.click(clear_all_goals, inputs=[], outputs=[goals_df]).then(
        check_goals_completion, inputs=[goals_df], outputs=[btn_goal_yes]
    )
    btn_download_csv.click(export_goals_csv, inputs=[goals_df], outputs=[btn_download_csv]) # 注意: DownloadButton 的 output 是它自己

    # 4. 目標達成/未達成 (舊有邏輯 + 按鈕狀態重置)
    btn_goal_yes.click(
        handle_goal_yes, inputs=None, outputs=[overlay_happy, overlay_angry, gacha_button], js=js_turn_light
    )
    btn_goal_no.click(
        handle_goal_no, inputs=None, outputs=[overlay_happy, overlay_angry, gacha_button], js=js_turn_dark
    )

    # 5. 其他原有功能
    btn_close_happy.click(close_overlays, inputs=None, outputs=[overlay_happy, overlay_angry])
    btn_close_angry.click(close_overlays, inputs=None, outputs=[overlay_happy, overlay_angry])
    diet_button.click(
        get_diet_suggestion,
        inputs=[character_selector, goals_df], # 👈 確保這裡傳入兩個元件
        outputs=[game_output]
    )
    gacha_button.click(get_gacha_reward, inputs=[character_selector], outputs=[game_output, heart_bar, plot_count, gacha_count_display, scene_image, gallery_display, secret_group, btn_floating_mail, secret_story_display])
    btn_close_secret.click(close_secret_letter, inputs=[], outputs=[secret_group, btn_floating_mail])
    btn_floating_mail.click(open_secret_letter, inputs=[], outputs=[secret_group, btn_floating_mail])

In [15]:
# 啟動 Gradio
demo.launch(debug=True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://b00a494eb701698131.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


DEBUG: 已載入 1 的歷史資料，共 1 筆。
DEBUG: 已載入 1 的歷史資料，共 1 筆。
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://b00a494eb701698131.gradio.live
